In [8]:
# the output directory for generator inference results on the test set
# traits = ['o', 'c', 'e', 'a', 'n']
traits = ['a', 'e']
in_dirs = {}
for trait in traits:
    in_dirs[trait] = f'/home/jiaruil5/personality/llm_personality/llm_bigfive/generator/outputs/test_{trait}/generated_predictions.jsonl'
in_dirs

{'a': '/home/jiaruil5/personality/llm_personality/llm_bigfive/generator/outputs/test_a/generated_predictions.jsonl',
 'e': '/home/jiaruil5/personality/llm_personality/llm_bigfive/generator/outputs/test_e/generated_predictions.jsonl'}

In [2]:
import pandas as pd
data_ref = pd.read_csv('/data/user_data/wenkail/llm_personality/data/big5_data_classifier_test.csv').to_dict(orient='records')

In [5]:
import json
import re
def convert_data(in_file):
    """in_file: innput file path, a jsonl file
    """
    data = [json.loads(i) for i in open(in_file, 'r').readlines()]
    pattern = r"Help me complete the sentence with certain Big Five Personality: (?:Openness|Conscientiousness|Extraversion|Agreeableness|Neuroticism) - (?:low|high)\n"
    
    data_lst = []
    for item, ref in zip(data, data_ref):
        # get label and input, output as a csv file
        # label: ope_z_label,con_z_label,ext_z_label,agr_z_label,neu_z_label
        # input: message
        message = item['prompt'].split("assistant\n\n")
        if len(message) == 1:
            continue
        message = re.split(pattern, message[0])[-1] + " " + item['predict']
        
        info = {
            "ope_z_label": ref['ope_z_label'],
            "con_z_label": ref['con_z_label'],
            "ext_z_label": ref['ext_z_label'],
            "agr_z_label": ref['agr_z_label'],
            "neu_z_label": ref['neu_z_label'],
            "message": message
        }
        data_lst.append(info)
    return data_lst

In [9]:
for trait in traits:
    res = convert_data(in_dirs[trait])
    df = pd.DataFrame().from_records(res)
    df.to_csv(f"/data/user_data/wenkail/llm_personality/generator/data/outputs/generator_predictions_{trait}.csv")

In [ ]:
# from utils import preprocess_function_with_tokenizer
# for split in ['train', 'val', 'test']:
#     df = pd.read_csv(f'/data/user_data/wenkail/llm_personality/data/big5_data_classifier_{split}.csv')
#     dataset = Dataset.from_pandas(df.dropna())
#     tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
#     dataset = dataset.map(lambda examples: preprocess_function_with_tokenizer(examples, tokenizer), batched=True)
#     dataset.save_to_disk(f'/data/user_data/wenkail/llm_personality/data_mse/{split}_psychgen')